In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

# if not os.getenv("OPENAI_API_KEY"):
#     raise ValueError("OPENAI_API_KEY is not set in .env file")
if not os.getenv("ANTHROPIC_API_KEY"):
    raise ValueError("ANTHROPIC_API_KEY is not set in .env file")
if not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("GOOGLE_API_KEY is not set in .env file")
if not os.getenv("DEEPSEEK_API_KEY"):
    raise ValueError("DEEPSEEK_API_KEY is not set in .env file")

In [2]:
import sys
import json
import asyncio
from typing import List, Dict, Any
from datetime import datetime
import pandas as pd

# LangChain関連
from langchain.llms import OpenAI
from langchain.schema import HumanMessage
from langchain.callbacks import get_openai_callback
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_deepseek import ChatDeepSeek
from langchain_google_genai import ChatGoogleGenerativeAI

# OpenAI Evals関連
from evals.eval import Eval
from evals.registry import Registry
from evals.data import get_jsonl

# Add the project root directory to Python path
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
sys.path.append(project_root)
from generator.generator import MahjongQuestionGenerator
from prompts.prompts import (
    generate_question_prompt_template,
    generate_question_with_mcp_prompt_template,
)
from evalutor.evals import MahjongEvaluator, MultiModelEvaluator

/Users/hiromu.nakamura.001/ghq/github.com/po3rin/llmmj/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Configure logging to see error messages
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

In [4]:
# モデル設定（APIキーが必要）
try:
    # # OpenAI GPT-4
    # gpt4 = ChatOpenAI(model_name="gpt-4o", temperature=0)

    # Claude (Anthropic APIキーが必要)
    claude_sonnet = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)
    claude_opus = ChatAnthropic(model="claude-opus-4-20250514", temperature=0)
    claude_3_7_sonnet = ChatAnthropic(model="claude-3-7-sonnet-20250219", temperature=0)

    # Google Gemini
    gemini = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-preview-05-20", temperature=0
    )

    # DeepSeek
    deepseek = ChatDeepSeek(model="deepseek-chat", temperature=0)

except Exception as e:
    print(f"モデル初期化エラー: {e}")
    raise e

In [5]:
# 評価システム初期化
generator = MahjongQuestionGenerator(
    model=claude_3_7_sonnet, query_template=generate_question_prompt_template
)

# Use the async version in Jupyter notebook
generator.generate_question(query="答えが2翻20符になる問題を作ってください")

{'tiles': ['1m',
  '2m',
  '3m',
  '4m',
  '5m',
  '6m',
  '7m',
  '8m',
  '9m',
  '2p',
  '3p',
  '4p',
  '5z',
  '5z'],
 'melds': None,
 'win_tile': '5z',
 'dora_indicators': ['4z'],
 'is_riichi': True,
 'is_tsumo': False,
 'is_ippatsu': False,
 'is_rinshan': False,
 'is_chankan': False,
 'is_haitei': False,
 'is_houtei': False,
 'is_daburu_riichi': False,
 'is_nagashi_mangan': False,
 'is_tenhou': False,
 'is_chiihou': False,
 'is_renhou': False,
 'is_open_riichi': False,
 'player_wind': 'east',
 'round_wind': 'east'}

In [6]:
# 評価システム初期化(with mcp)
generator = MahjongQuestionGenerator(
    model=claude_3_7_sonnet, use_mcp=True, query_template=generate_question_with_mcp_prompt_template
)

# Use the async version in Jupyter notebook
generator.generate_question(query="答えが2翻20符になる問題を作ってください")

{'tiles': ['1m',
  '2m',
  '3m',
  '4m',
  '5m',
  '6m',
  '7m',
  '8m',
  '9m',
  '2p',
  '3p',
  '4p',
  '5z',
  '5z'],
 'melds': None,
 'win_tile': '5z',
 'dora_indicators': ['4z'],
 'is_riichi': True,
 'is_tsumo': False,
 'is_ippatsu': False,
 'is_rinshan': False,
 'is_chankan': False,
 'is_haitei': False,
 'is_houtei': False,
 'is_daburu_riichi': False,
 'is_nagashi_mangan': False,
 'is_tenhou': False,
 'is_chiihou': False,
 'is_renhou': False,
 'is_open_riichi': False,
 'player_wind': 'east',
 'round_wind': 'east'}

In [7]:
# dataset = [
#     {
#         "query": "答えが2翻40符になる問題を作ってください",
#         "answer": {
#             "fu": 40,
#             "han": 2
#         }
#     }
# ]

In [8]:
# evaluator = MahjongEvaluator(generator=generator)
# df = evaluator.evals(dataset=dataset)
# df

In [9]:
# e = MultiModelEvaluator(models=[claude_sonnet], query_template=generate_question_prompt_template)
# df = e.evals(dataset)
# df

In [10]:
with open("../dataset/queries.min.json", "r") as f:
    dataset = json.load(f)

In [11]:
# dataset

In [12]:
models = {
    # "claude_sonnet": claude_sonnet,
    # "claude_opus": claude_opus,
    # "gemini": gemini,
    # "deepseek": deepseek
}

In [13]:
for k, m in models.items():
    e = MultiModelEvaluator(
        models=[m], query_template=generate_question_prompt_template
    )
    df = e.evals(dataset)

    df.to_csv(
        f"../dist/evals-gen-question-raw-prompt-{k}-{datetime.now().strftime('%Y%m%d')}.csv",
        index=False,
    )
    print(f"{k} done :)")